In [2]:
import pandas as pd
data = pd.read_csv('medical_data.csv')
test_data = data[['death', 'Na+', 'DBP', 'PLT', 'ivabradine', 'MRA' ]]

In [4]:
import pandas as pd
from scipy import stats 

def perform_tests(data):
    mann_whitney = []
    ttest = []
    chi_square = []
    shapiro_wilk = []
    for col in data.columns:
        if col == 'death':
            continue
        if data[col].dtype == 'int64':
            temp = pd.crosstab(data[col], data['death'])
            _, p, _, _ = stats.chi2_contingency(temp)
            chi_square.append((col, p.round(4)))
        else:
            _, p0 = stats.shapiro(data[data.death == 0][col])
            _, p1 = stats.shapiro(data[data.death == 1][col])
            shapiro_wilk.append((col, (round(p0, 4), round(p1, 4))))
            if p0 > 0.05 and p1 > 0.05:
                _, p = stats.ttest_ind(a=data[data.death==0][col], b=data[data.death==1][col], equal_var=False)
                ttest.append((col, p.round(4)))
                
            else:
                _, p = stats.mannwhitneyu(data[data.death == 1][col], data[data.death == 0][col], alternative='greater')
                mann_whitney.append((col, p.round(4)))
    
    # Burasi bence zorunlu degil ama sonucta gosterilen sirada dondurmek istiyorsan p-value degerine gore dusukten yuksege siralama...
    mann_whitney.sort(key=lambda x: x[1])
    ttest.sort(key=lambda x: x[1])
    chi_square.sort(key=lambda x: x[1])
    shapiro_wilk.sort(key=lambda x: x[1])


    return {'mann_whitney' : mann_whitney,
    'ttest': ttest,
    'chi_square': chi_square,
    'shapiro_wilk': shapiro_wilk}

perform_tests(test_data)

{'mann_whitney': [('Na+', 0.2143)],
 'ttest': [('DBP', 0.0), ('PLT', 0.4739)],
 'chi_square': [('ivabradine', 0.0144), ('MRA', 0.2884)],
 'shapiro_wilk': [('Na+', (0.0, 0.0071)),
  ('PLT', (0.2361, 0.6935)),
  ('DBP', (0.5272, 0.3715))]}